In [5]:
import os
import jieba
from sklearn.datasets.base import Bunch
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer#feature_extraction‘特征提取’,Convert a collection of raw documents to a matrix of TF-IDF features.

'''
The sklearn.feature_extraction module deals with feature extraction from raw data. 
It currently includes methods to extract features from text and images.
The sklearn.feature_extraction.text submodule gathers utilities to build feature vectors from text documents.
'''

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB #朴素贝叶斯分类算法，Naive Bayes classifier for multinomial models

'''
The multinomial Naive Bayes classifier is suitable for classification with discrete features
 (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts.
  However, in practice, fractional counts such as tf-idf may also work.
'''

from sklearn.metrics import classification_report#metrics度量，分类报告Build a text report showing the main classification metrics

In [6]:
with open('./cnews.jieba.txt','r',encoding='utf-8') as f:
    contents = []
    lines=f.readlines()
    for line in lines:
        category,content=line.strip('\r\n').split('\t')#由于wordli又通过\t和\n将分类和对应的新闻又组合在了一起
        contents.append(content)

In [7]:
print(len(contents))
print(contents[0])

10000
鲍勃 库西 奖归 属 NCAA 最强 控卫 坎巴 弗神 新浪 体育讯 本赛季 NCAA 进入 末段 各项 奖项 评选 出炉 评选 最佳 控卫 鲍勃 库西 奖 下周 最终 四强 战时 公布 鲍勃 库西 奖是 由奈 史密斯 篮球 名人堂 提供 旨在 奖励 年度 最佳 大学 控卫 最终 获奖 球员 以下 几名 热门 人选 中 产生 NCAA 疯狂 三月 专题 主页 上线 点击 链接 查看 精彩内容 吉梅尔 弗雷 戴特 杨百翰 大学 弗神 吉梅尔 弗雷 戴特 一直 备受 关注 一名 射手 他会用 终结 对手 脚踝 变向 掉 面前 防守 任意 一支 手 完成 得分 犯规 提前 两份 划入 帐 下 一名 命中率 高达 90% 罚球 手 弗雷 戴特 具有 所有 伟大 控卫 具备 一点 特质 一位 赢家 一位 领导者 整个 赛季 至始 至终 稳定 领导 球队 前进 这是 无可比拟 杨百翰 大学 主教练 戴夫 罗斯 称赞 道 得分 能力 毋庸置疑 认为 带领 球队 获胜 能力 最 重要 控卫 职责 主场 之外 比赛 客场 中 立场 取胜 19 场 表现 很棒 弗雷 戴特 能否 NBA 取得成功 专业人士 资格 做出 判断 喜爱 凯尔特人 主教练 多克 里 弗斯 说道 很棒 看过 ESPN 片段 剪辑 剪辑 超级 巨星 认为 成为 一名 优秀 NBA 球员 诺兰 史密斯 杜克大学 赛季 球队 宣布 一天 控卫凯瑞 厄尔 文因 脚趾 伤病 缺席 赛季 大部分 比赛 后 诺兰 史密斯 接管 球权 进攻 端上 足 发条 ACC 联盟 杜克大学 所在 分区 得分 榜上 名列前茅 分区 助攻 榜上 占据 头名 众强 林立 ACC 联盟 前无古人 认为 全美 球员 凯瑞 厄尔 文 受伤 后 好 接管 球队 之前 毫无准备 杜克 主教练 迈克 沙舍 夫斯基 赞扬 道 他会 比赛 带入 节奏 得分 组织 领导 球队 无所不能 现在 攻防 俱佳 持球 人 防守 提高 拥有 辉煌 赛季 坎巴 沃克 康涅狄格 大学 坎巴 沃克 带领 康涅狄格 赛季 毛伊岛 邀请赛 一路 力克 密歇根州 肯塔基 等队 夺冠 他场 30 分 助攻 得到 最佳 球员 大东 赛区 锦标赛 全国 锦标赛 中 他场 27.1 分 6.1 篮板 5.1 次 助攻 依旧 给力 疯狂 表现 赛季 疯狂 表现 结束 赛季 全国 锦

## ***   
[CounVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)的API请自行参考sklearn，文中代码限定term出现次数必须大于2，最终保留前n_features=2500的term作为features。训练得到的tf_vectorizer 利用joblib保存到文件，第二次起可以直接从文件中load进来避免重复计算。该步骤得到的tf矩阵为一个“文章-词语”稀疏矩阵，可以通过tf_vectorizer.get_feature_names()得到每一维feature对应的term。    

[文章和代码参考](https://blog.csdn.net/TiffanyRabbit/article/details/76445909)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib  #也可以选择pickle等保存模型，请随意

n_features = 5000

# tf_vector = TfidfVectorizer(max_df=0.95, min_df=2,
#                                 max_features=n_features)
#构建词汇统计向量并保存，仅运行首次
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features)

tf = tf_vectorizer.fit_transform(contents)

In [9]:
from sklearn.decomposition import LatentDirichletAllocation
n_topic = 10
lda = LatentDirichletAllocation(n_topics=n_topic, 
                                max_iter=50,
                                learning_method='batch')
lda.fit(tf) #tf即为Document_word Sparse Matrix       

D:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# #  权重策略TF——IDF

* TF-IDF(Term frequency * Inverse Doc Frequency)词权重  

* 在较低的文本语料库中，一些词非常常见（例如，英文中的“the”，“a”，“is”），因此很少带有文档实际内容的有用信息。   

* 如果我们将单纯的计数数据直接喂给分类器，那些频繁出现的词会掩盖那些很少出现但是更有意义的词的频率。  
****
* 为了重新计算特征的计数权重，以便转化为适合分类器使用的浮点值，通常都会进行tf-idf转换。  

* 词重要性度量一般使用文本挖掘的启发式方法：TF-IDF。
 这是一个最初为信息检索（作为搜索引擎结果的排序功能）开发的词加权机制，在文档分类和聚类中也是非常有用的   

* 由于tf-idf经常用于文本特征，因此有另一个类称为TfidfVectorizer，

* 将CountVectorizer和TfidfTransformer的所有选项合并在一个模型中



**enumerate()** 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。     
seq = ['one', 'two', 'three']    
for i, element in enumerate(seq):   
   print i, element   
 
0 one   
1 two  
2 three  


In [12]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic #%d:" % topic_idx)
        print (" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
                        
    print
    #打印主题-词语分布矩阵
    print (model.components_)

n_top_words=20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
中国 发展 问题 工作 经济 国家 表示 进行 合作 建设 政府 国际 加强 部门 规定 社会 记者 要求 相关 会议
Topic #1:
搭配 时尚 设计 风格 组图 性感 黑色 选择 造型 导语 白色 感觉 外套 流行 颜色 款式 气质 色彩 明星 装扮
Topic #2:
功能 像素 采用 拍摄 支持 机身 英寸 镜头 佳能 光学 相机 拥有 索尼 高清 自动 对焦 性能 具有 使用 编辑
Topic #3:
电影 一个 导演 微博 新浪 影片 觉得 表示 中国 记者 现在 观众 娱乐 拍摄 已经 票房 希望 角色 演员 知道
Topic #4:
产品 企业 家具 一个 品牌 市场 消费者 行业 中国 家居 发展 公司 装修 一些 设计 服务 现在 销售 问题 环保
Topic #5:
学生 留学 美国 移民 大学 中国 申请 学校 教育 学习 专业 考试 一个 签证 留学生 英国 孩子 工作 选择 课程
Topic #6:
比赛 球队 篮板 球员 季后赛 时间 赛季 热火 火箭 新浪 进攻 已经 防守 湖人 一个 命中 体育讯 助攻 表现 三分
Topic #7:
房地产 市场 项目 房价 地板 北京 价格 土地 记者 今年 开发商 楼市 城市 政策 投资 地产 已经 10 目前 平方米
Topic #8:
基金 投资 公司 市场 股票 指数 型基金 投资者 收益 债券 2008 经理 发行 分红 去年 资产 上涨 净值 管理 显示
Topic #9:
游戏 玩家 活动 手机 ol 任务 获得 网游 奖励 一个 装备 系统 10 世界 客服 经验 online com 体验 时间
[[  1.11518712e+01   1.00033790e-01   1.00010347e-01 ...,   1.00000449e-01
    2.14160088e+02   6.78202550e-01]
 [  1.00012526e-01   1.67909408e+01   1.00006393e-01 ...,   1.00018562e-01
    1.00008002e-01   1.00018207e-01]
 [  5.88283370e+00   1.88914350e+01   1.00000934e-01 ...,   

## 参数
Class sklearn.decomposition.LatentDirichletAllocation(n_topics=10, doc_topic_prior=None, topic_word_prior=None, learning_method=None, learning_decay=0.7, learning_offset=10.0, max_iter=10, batch_size=128, evaluate_every=-1, total_samples=1000000.0, perp_tol=0.1, mean_change_tol=0.001, max_doc_update_iter=100, n_jobs=1, verbose=0, random_state=None)   
--------------------- 

参数： 
1) n_topics: 即我们的隐含主题数K,需要调参。K的大小取决于我们对主题划分的需求，比如我们只需要类似区分是动物，植物，还是非生物这样的粗粒度需求，那么K值可以取的很小，个位数即可。如果我们的目标是类似区分不同的动物以及不同的植物，不同的非生物这样的细粒度需求，则K值需要取的很大，比如上千上万。此时要求我们的训练文档数量要非常的多。    
2) doc_topic_prior:即我们的文档主题先验Dirichlet分布θd的参数α。一般如果我们没有主题分布的先验知识，可以使用默认值1/K。    
3) topic_word_prior:即我们的主题词先验Dirichlet分布βk的参数η。一般如果我们没有主题分布的先验知识，可以使用默认值1/K。    
4) learning_method: 即LDA的求解算法。有 ‘batch’ 和 ‘online’两种选择。 ‘batch’即我们在原理篇讲的变分推断EM算法，而”online”即在线变分推断EM算法，在”batch”的基础上引入了分步训练，将训练样本分批，逐步一批批的用样本更新主题词分布的算法。默认是”online”。选择了‘online’则我们可以在训练时使用partial_fit函数分布训练。不过在scikit-learn 0.20版本中默认算法会改回到”batch”。建议样本量不大只是用来学习的话用”batch”比较好，这样可以少很多参数要调。而样本太多太大的话，”online”则是首先了。    
5）learning_decay：仅仅在算法使用”online”时有意义，取值最好在(0.5, 1.0]，以保证”online”算法渐进的收敛。主要控制”online”算法的学习率，默认是0.7。一般不用修改这个参数。     
6）learning_offset：仅仅在算法使用”online”时有意义，取值要大于1。用来减小前面训练样本批次对最终模型的影响。   
7）max_iter ：EM算法的最大迭代次数。    
8）total_samples：仅仅在算法使用”online”时有意义， 即分步训练时每一批文档样本的数量。在使用partial_fit函数时需要。   
9）batch_size: 仅仅在算法使用”online”时有意义， 即每次EM算法迭代时使用的文档样本的数量。   
10）mean_change_tol :即E步更新变分参数的阈值，所有变分参数更新小于阈值则E步结束，转入M步。一般不用修改默认值。   
11） max_doc_update_iter: 即E步更新变分参数的最大迭代次数，如果E步迭代次数达到阈值，则转入M步。  

方法：    
1）fit(X[, y])：利用训练数据训练模型，输入的X为文本词频统计矩阵。   
2）fit_transform(X[, y])：利用训练数据训练模型，并返回训练数据的主题分布。   
3）get_params([deep])：获取参数   
4）partial_fit(X[, y])：利用小batch数据进行Online方式的模型训练。   
5）perplexity(X[, doc_topic_distr, sub_sampling])：计算X数据的approximate perplexity。   
6）score(X[, y])：计算approximate log-likelihood。   
7）set_params(params)：设置参数。    
8）transform(X)：利用已有模型得到语料X中每篇文档的主题分布。 
